In [5]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
import folium
from sklearn import preprocessing
import geopandas as gpd
import matplotlib.colors as colors
import matplotlib.colors as colors
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from helpers import *
from ipywidgets import SelectMultiple, RadioButtons, interact, Layout
%load_ext autoreload
%autoreload 2

## Trade Network

In [6]:
import networkx as nx

In [7]:
# load trade matrix
mat = pd.read_pickle("pickle/trade_matrix.pkl")

In [8]:
# nodes
countries = set(mat.reset_index().From.values).union(mat.reset_index().To.values)

### Graph 1: Raw weights

In [9]:
G1 = nx.DiGraph()

In [10]:
G1.add_nodes_from(countries)

In [11]:
# weighted edges
edges = mat.mean(axis=1)

In [12]:
G1.add_weighted_edges_from(edges.reset_index().values)

In [13]:
# compute eigenvector centrality
eig1 = nx.eigenvector_centrality_numpy(G1,'weight')
eig1 = pd.Series(eig1).sort_values(ascending=False)

In [14]:
eig1.head(10)

United States of America    0.421350
Germany                     0.380337
Japan                       0.327307
China                       0.282327
France                      0.271977
United Kingdom              0.264886
Canada                      0.253817
Netherlands                 0.242446
Italy                       0.221345
Mexico                      0.215367
dtype: float64

### Graph 2: Normalized weigths

In [15]:
# load production value
prodC = pd.read_pickle("pickle/value_production_crops.pkl")
prodL = pd.read_pickle("pickle/value_production_livestock.pkl")

In [16]:
# total production in recent years
years = ['Y'+str(2000+x) for x in range(0,17)]
production = prodC[years]+prodL[years]
prod_mean = production.mean(axis=1)

In [17]:
production = production.join(mat.reset_index().set_index('From').To).reset_index()
production = production.rename(columns={'index':'From'}).set_index(['From','To'])
production = production.loc[mat.index,:]

In [18]:
mat_norm = mat[years]/production
mat_norm.head()

Y2000     Y2001     Y2002     Y2003     Y2004  \
From        To                                                           
Afghanistan China     0.001052  0.000027  0.000014  0.000102  0.000158   
            Germany   0.000005  0.000044  0.000007  0.000056  0.000041   
            India     0.004965  0.003973  0.003074  0.005618  0.006237   
            Iran      0.000181  0.000025  0.000020  0.000129  0.000429   
            Pakistan  0.004941  0.003550  0.002952  0.003747  0.004649   

                         Y2005     Y2006     Y2007     Y2008     Y2009  \
From        To                                                           
Afghanistan China     0.000215  0.000005  0.000278  0.000379  0.000629   
            Germany   0.000113  0.000154  0.000692  0.000273  0.000581   
            India     0.007958  0.003244  0.012292  0.018754  0.024749   
            Iran      0.000550  0.000000  0.000000  0.000000  0.004755   
            Pakistan  0.001918  0.004090  0.007469  0.007269  0.024448   

                         Y2010     Y2011     Y2012     Y2013     Y2014  \
From        To                                                           
Afghanistan China     0.001879  0.001102  0.000391  0.001088  0.003774   
            Germany   0.002753  0.001340  0.000345  0.000798  0.002290   
            India     0.020296  0.023685  0.005593  0.024300  0.045046   
            Iran      0.005876  0.002492  0.000000  0.000000  0.003531   
            Pakistan  0.023368  0.020316  0.016391  0.026539  0.043628   

                         Y2015     Y2016  
From        To                            
Afghanistan China     0.001486  0.000140  
            Germany   0.001009  0.000795  
            India     0.065573  0.053363  
            Iran      0.000000  0.004435  
            Pakistan  0.070727  0.059137

In [19]:
mat_norm.to_pickle("pickle/trade_matrix_norm.pkl")

In [20]:
G2 = nx.DiGraph()

In [21]:
G2.add_nodes_from(countries)

In [22]:
# weighted edges
edges = mat_norm.mean(axis=1)

In [23]:
G2.add_weighted_edges_from(edges.reset_index().values)

In [24]:
# compute eigenvector centrality
eig2 = nx.eigenvector_centrality_numpy(G2,'weight')
eig2 = pd.Series(eig2).sort_values(ascending=False)

In [25]:
eig2.head(10)

Germany                     0.615902
France                      0.424744
United Kingdom              0.344476
Netherlands                 0.289199
Italy                       0.253267
Belgium                     0.214993
United States of America    0.185938
Spain                       0.148625
Austria                     0.088384
Poland                      0.083301
dtype: float64

In [26]:
pd.Series(eig2*prod_mean).sort_values(ascending=False).head(10)

United States of America    4.269975e+07
China                       2.832978e+07
Germany                     2.192796e+07
France                      1.773009e+07
Italy                       8.007288e+06
United Kingdom              6.161696e+06
Spain                       4.900980e+06
Netherlands                 3.735338e+06
Russia                      3.665178e+06
India                       1.840024e+06
dtype: float64